# 近隣の場所を検索するための関数呼び出し：Google Places APIと顧客プロファイルの活用

このノートブックは、Google Places APIとカスタムユーザープロファイルの統合に焦点を当て、位置情報ベースの検索を強化することを目的としています。私たちのアプローチは、Google Places APIをユーザーの好みと組み合わせて使用し、場所の発見をよりパーソナルで関連性の高いものにすることを目指しています。この例ではGoogle Places APIに焦点を当てていますが、同様の方法で探索・適用できる他の多くのAPIが存在することにご注意ください。

私たちは3つの主要なコンポーネントの応用を探求します：

- 顧客プロファイル：このモックプロファイルは、場所のタイプ（例：レストラン、公園、博物館）、予算、好みの評価、その他の特定の要件に対する個人の好みを捉えます。

- Google Places API：このAPIは近隣の場所に関するリアルタイムデータを提供します。評価、会場のタイプ、コスト、その他の周辺の場所からの様々なデータポイントを考慮します。

- 関数呼び出し：「お腹が空いた」や「博物館を訪れたい」などの単一のコマンドが関数を起動し、ユーザープロファイルデータとGoogle Places APIを組み合わせて適切な会場を特定します。

このノートブックでは、2つの主要なユースケースを紹介します：

- プロファイルベースの推奨：ユーザープロファイルを作成し、個人の好みに基づいて場所の推奨を行う方法を学びます。

- 関数呼び出しによるAPI統合：関数呼び出しを使用してGoogle Places APIを効果的に統合・呼び出し、様々な場所のリアルタイムデータを取得する方法を理解します。

このシステムは非常に汎用性が高いものの、その効果はユーザーの好みや利用可能な場所データによって変わる可能性があることにご注意ください。このノートブックの目的上、顧客データは偽物であり、場所はハードコードされています。

## セットアップ

Google Places API

Google Places APIを使用するには、以下の2つが必要です：

- Googleアカウント：まだお持ちでない場合は、Googleアカウントを作成する必要があります。

- Google Places API キー：APIキーは、使用量と課金の目的でプロジェクトに関連付けられたリクエストを認証するために使用される一意の識別子です。APIキーは[Google Cloud Console](https://console.cloud.google.com/getting-started?authuser=1)から取得できます。



Google Places APIは有料サービスであり、コストはAPI呼び出し回数に関連付けられていることにご注意ください。予期しない料金を避けるため、使用量を追跡してください。



requestsライブラリも必要です。以下のコマンドを使用してダウンロードできます：

```python
pip install requests
```

In [1]:
import json
from openai import OpenAI
import os
import requests

client = OpenAI(api_key=os.environ.get("OPENAI_API_KEY", "<your OpenAI API key if not set as env var>"))

このコードスニペットでは、`user_id`を受け取り、モックユーザープロファイルを返す`fetch_customer_profile`関数を定義しています。

この関数は、データベースからユーザーデータを取得するAPI呼び出しをシミュレートしています。このデモでは、ハードコードされたデータを使用しています。ユーザープロファイルには、ユーザーの位置情報（この例ではゴールデンゲートブリッジの座標に設定）、食べ物やアクティビティの好み、アプリ使用状況メトリクス、最近のインタラクション、ユーザーランクなど、さまざまな詳細情報が含まれています。

本番環境では、このハードコードされたデータを実際のユーザーデータベースへのAPI呼び出しに置き換えることになります。

In [2]:
def fetch_customer_profile(user_id):
    # You can replace this with a real API call in the production code
    if user_id == "user1234":
        return {
            "name": "John Doe",
            "location": {
                "latitude": 37.7955,
                "longitude": -122.4026,
            },
            "preferences": {
                "food": ["Italian", "Sushi"],
                "activities": ["Hiking", "Reading"],
            },
            "behavioral_metrics": {
                "app_usage": {
                    "daily": 2,  # hours
                    "weekly": 14  # hours
                },
                "favourite_post_categories": ["Nature", "Food", "Books"],
                "active_time": "Evening",
            },
            "recent_searches": ["Italian restaurants nearby", "Book clubs"],
            "recent_interactions": ["Liked a post about 'Best Pizzas in New York'", "Commented on a post about 'Central Park Trails'"],
            "user_rank": "Gold",  # based on some internal ranking system
        }
    else:
        return None


## Google Places APIからのデータ要求と処理

関数`call_google_places_api`は、Google Places APIから情報を要求し、指定された`place_type`とオプションの`food_preference`に基づいて上位2つの場所のリストを提供する役割を果たします。これは有料サービスであるため、使用量を管理するためにこの関数を上位2つの結果に制限しています。ただし、要件に応じて任意の数の結果を取得するように変更することができます。

この関数は、ハードコードされた場所（トランスアメリカピラミッドの座標に設定）、あなたのGoogle APIキー、および特定のリクエストパラメータで設定されています。`place_type`に応じて、適切なAPIリクエストURLを作成します。`place_type`がレストランで`food_preference`が指定されている場合、それはAPIリクエストに含まれます。

GETリクエストを送信した後、関数はレスポンスステータスをチェックします。成功した場合、JSONレスポンスを処理し、`get_place_details`関数を使用して関連する詳細を抽出し、人間が読みやすい形式で返します。リクエストが失敗した場合、デバッグのためにエラーを出力します。

`get_place_details`関数は、指定された`place_id`を使用して場所のより詳細な情報を取得するために使用されます。Google Place Details APIにGETリクエストを送信し、リクエストが成功した場合は結果を返します。リクエストが失敗した場合、デバッグのためにエラーを出力します。

両方の関数は例外を処理し、何か問題が発生した場合はエラーメッセージを返します。

In [3]:
def get_place_details(place_id, api_key):
    URL = f"https://maps.googleapis.com/maps/api/place/details/json?place_id={place_id}&key={api_key}"
    response = requests.get(URL)
    if response.status_code == 200:
        result = json.loads(response.content)["result"]
        return result
    else:
        print(f"Google Place Details API request failed with status code {response.status_code}")
        print(f"Response content: {response.content}")
        return None


In [4]:
def call_google_places_api(user_id, place_type, food_preference=None):
    try:
        # Fetch customer profile
        customer_profile = fetch_customer_profile(user_id)
        if customer_profile is None:
            return "I couldn't find your profile. Could you please verify your user ID?"

        # Get location from customer profile
        lat = customer_profile["location"]["latitude"]
        lng = customer_profile["location"]["longitude"]

        API_KEY = os.getenv('GOOGLE_PLACES_API_KEY')  # retrieve API key from environment variable
        LOCATION = f"{lat},{lng}"
        RADIUS = 500  # search within a radius of 500 meters
        TYPE = place_type

        # If the place_type is restaurant and food_preference is not None, include it in the API request
        if place_type == 'restaurant' and food_preference:
            URL = f"https://maps.googleapis.com/maps/api/place/nearbysearch/json?location={LOCATION}&radius={RADIUS}&type={TYPE}&keyword={food_preference}&key={API_KEY}"
        else:
            URL = f"https://maps.googleapis.com/maps/api/place/nearbysearch/json?location={LOCATION}&radius={RADIUS}&type={TYPE}&key={API_KEY}"

        response = requests.get(URL)
        if response.status_code == 200:
            results = json.loads(response.content)["results"]
            places = []
            for place in results[:2]:  # limit to top 2 results
                place_id = place.get("place_id")
                place_details = get_place_details(place_id, API_KEY)  # Get the details of the place

                place_name = place_details.get("name", "N/A")
                place_types = next((t for t in place_details.get("types", []) if t not in ["food", "point_of_interest"]), "N/A")  # Get the first type of the place, excluding "food" and "point_of_interest"
                place_rating = place_details.get("rating", "N/A")  # Get the rating of the place
                total_ratings = place_details.get("user_ratings_total", "N/A")  # Get the total number of ratings
                place_address = place_details.get("vicinity", "N/A")  # Get the vicinity of the place

                if ',' in place_address:  # If the address contains a comma
                    street_address = place_address.split(',')[0]  # Split by comma and keep only the first part
                else:
                    street_address = place_address

                # Prepare the output string for this place
                place_info = f"{place_name} is a {place_types} located at {street_address}. It has a rating of {place_rating} based on {total_ratings} user reviews."

                places.append(place_info)

            return places
        else:
            print(f"Google Places API request failed with status code {response.status_code}")
            print(f"Response content: {response.content}")  # print out the response content for debugging
            return []
    except Exception as e:
        print(f"Error during the Google Places API call: {e}")
        return []


## GPT-3.5-TurboとGoogle Places APIを使用したユーザー固有の推奨事項の生成

関数`provide_user_specific_recommendations`は、GPT-3.5-TurboとGoogle Places APIと連携して、ユーザーの好みと位置情報に合わせたレスポンスを提供します。

まず、`user_id`を使用して顧客のプロフィールを取得します。プロフィールが見つからない場合は、エラーメッセージを返します。

有効なプロフィールがある場合、顧客の食べ物の好みを抽出し、OpenAIモデルと連携します。AIモデルに対して初期のシステムメッセージを提供し、その役割、ユーザーの好み、Google Places API関数の使用方法についてのコンテキストを与えます。

ユーザーの入力もメッセージとしてモデルに送信され、関数`call_google_places_api`が`functions`パラメータで定義されて、AIモデルが必要に応じて呼び出せるようになっています。

最後に、モデルのレスポンスを処理します。モデルがGoogle Places APIへの関数呼び出しを行う場合、適切な引数で関数が実行され、近くの場所の名前が返されます。そのような場所がない場合やリクエストが理解されない場合は、適切なエラーメッセージが返されます。

In [7]:
def provide_user_specific_recommendations(user_input, user_id):
    customer_profile = fetch_customer_profile(user_id)
    if customer_profile is None:
        return "I couldn't find your profile. Could you please verify your user ID?"

    customer_profile_str = json.dumps(customer_profile)

    food_preference = customer_profile.get('preferences', {}).get('food', [])[0] if customer_profile.get('preferences', {}).get('food') else None


    response = client.chat.completions.create(
        model="gpt-3.5-turbo",
        messages=[
    {
        "role": "system",
        "content": f"You are a sophisticated AI assistant, a specialist in user intent detection and interpretation. Your task is to perceive and respond to the user's needs, even when they're expressed in an indirect or direct manner. You excel in recognizing subtle cues: for example, if a user states they are 'hungry', you should assume they are seeking nearby dining options such as a restaurant or a cafe. If they indicate feeling 'tired', 'weary', or mention a long journey, interpret this as a request for accommodation options like hotels or guest houses. However, remember to navigate the fine line of interpretation and assumption: if a user's intent is unclear or can be interpreted in multiple ways, do not hesitate to politely ask for additional clarification. Make sure to tailor your responses to the user based on their preferences and past experiences which can be found here {customer_profile_str}"
    },
    {"role": "user", "content": user_input}
],
        temperature=0,
        tools=[
            {
                "type": "function",
                "function" : {
                    "name": "call_google_places_api",
                    "description": "This function calls the Google Places API to find the top places of a specified type near a specific location. It can be used when a user expresses a need (e.g., feeling hungry or tired) or wants to find a certain type of place (e.g., restaurant or hotel).",
                    "parameters": {
                        "type": "object",
                        "properties": {
                            "place_type": {
                                "type": "string",
                                "description": "The type of place to search for."
                            }
                        }
                    },
                    "result": {
                        "type": "array",
                        "items": {
                            "type": "string"
                        }
                    }
                }
            }
        ],
    )

    print(response.choices[0].message.tool_calls)

    if response.choices[0].finish_reason=='tool_calls':
        function_call = response.choices[0].message.tool_calls[0].function
        if function_call.name == "call_google_places_api":
            place_type = json.loads(function_call.arguments)["place_type"]
            places = call_google_places_api(user_id, place_type, food_preference)
            if places:  # If the list of places is not empty
                return f"Here are some places you might be interested in: {' '.join(places)}"
            else:
                return "I couldn't find any places of interest nearby."

    return "I am sorry, but I could not understand your request."



## ユーザー固有の推奨事項の実行

実行時、この関数はユーザーのプロフィールを取得し、AIモデルと対話し、モデルの応答を処理し、必要に応じてGoogle Places APIを呼び出し、最終的にユーザーの好みと場所に合わせた推奨事項のリストを返します。出力される内容は、これらのパーソナライズされた推奨事項で構成されます。

In [8]:
user_id = "user1234"
user_input = "I'm hungry"
output = provide_user_specific_recommendations(user_input, user_id)
print(output)


[ChatCompletionMessageToolCall(id='call_Q1mXIi7D6GhobfE4tkruX7nB', function=Function(arguments='{\n  "place_type": "restaurant"\n}', name='call_google_places_api'), type='function')]
Here are some places you might be interested in: Sotto Mare is a restaurant located at 552 Green Street. It has a rating of 4.6 based on 3765 user reviews. Mona Lisa Restaurant is a restaurant located at 353 Columbus Avenue #3907. It has a rating of 4.4 based on 1888 user reviews.
